# Bring Your Own Checkpoint (BYOC)

Welcome to this tutorial on bringing your own checkpoint (BYOC) to SambaNova Cloud!

Please note that this functionality is only available to customers of the SambaNova Cloud dedicated tier. To run this notebook, you will need to set up the SambaNova SDK (SNSDK) and command line interface (SNAPI). 

1.  Install the following libraries using the wheels provided to you by your SambaNova representative:  
    - `pip install ~/Downloads/snsdk-<version>-py3-none-any.whl`  
    - `pip install ~/Downloads/SambaStudio_API_Client-<version>-py3-none-any.whl`

2.  Set the following environment variables in your `.env` file as provided by your SambaNova representative:
```  
SAMBASTUDIO_HOST_NAME = '<host name without any trailing "/">'
SAMBASTUDIO_ACCESS_KEY = '<access key>'
SAMBASTUDIO_TENANT_NAME = '<tenant name if provided, else "default">'

## 1.  Imports

In [1]:
import sys
sys.version

'3.11.11 (main, Dec 11 2024, 10:28:39) [Clang 14.0.6 ]'

In [2]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
import json
import os
from dotenv import load_dotenv
from huggingface_hub import hf_hub_download, HfApi
from pathlib import Path
import pprint
load_dotenv()

True

In [3]:
ai_starter_kit_path = "../"
byoc_path = ai_starter_kit_path + "utils/byoc"
sys.path.append(byoc_path)
from src.snsdk_byoc_wrapper import BYOC

## 2. Download checkpoint from HuggingFace

In [4]:
model_name = "Salesforce/Llama-xLAM-2-8b-fc-r"  # Replace with your desired model
home_dir = os.path.expanduser("~")
target_dir = os.path.join(home_dir + "/models")  
os.makedirs(target_dir, exist_ok=True)

repo_files = HfApi().list_repo_files(model_name)
for file_name in repo_files:
    if file_name == "config.json":
        config_path = hf_hub_download(repo_id=model_name, filename=file_name, cache_dir=target_dir)

## 3. Set your BYOC config

### Model metadata

The metadata values can be modified manually. Note that the `param_count` is hardcoded because it does not exist in any of the HF checkpoint configs.

In [5]:
byoc = BYOC()
byoc.config = {
    "checkpoints": [{
        "model_name": model_name.replace('/','--'),
        "checkpoint_path": str(Path(config_path).parent),
        "publisher": model_name.split('/')[0],
        "description": "BYOC - " + model_name,
        "param_count": 8 # number in billions of parameters
    }]
}

2025-05-02 09:22:31,212 [INFO] Using env variables to set up Snsdk and Snapi.


In [6]:
str(Path(config_path).parent)

'/Users/varunk/models/models--Salesforce--Llama-xLAM-2-8b-fc-r/snapshots/d10ff9682648d264a364a61505af5c0e172895f8'

### Model parameters

In [7]:
byoc.find_config_params()

2025-05-02 09:22:32,266 [INFO] Params for checkpoint in /Users/varunk/models/models--Salesforce--Llama-xLAM-2-8b-fc-r/snapshots/d10ff9682648d264a364a61505af5c0e172895f8:
[{'model_arch': 'llama', 'seq_length': 131072, 'vocab_size': 128256}]
2025-05-02 09:22:32,268 [INFO] config updated with checkpoints parameters


[{'model_arch': 'llama', 'seq_length': 131072, 'vocab_size': 128256}]

### Model application

In [8]:
suitable_apps = byoc.get_suitable_apps()[0]
suitable_apps

2025-05-02 09:22:33,886 [INFO] Checkpoint Salesforce--Llama-xLAM-2-8b-fc-r suitable apps:
[{'id': 'eb0aaad1-694f-41b6-958a-b974737635c4', 'name': 'Samba1 Llama3.1 Experts'}]


[{'id': 'eb0aaad1-694f-41b6-958a-b974737635c4',
  'name': 'Samba1 Llama3.1 Experts'}]

In [9]:
byoc.config["checkpoints"][0]["app_id"] = suitable_apps[0]["id"]

### Config summary

In [10]:
pprint.pprint(byoc.config)

{'checkpoints': [{'app_id': 'eb0aaad1-694f-41b6-958a-b974737635c4',
                  'checkpoint_path': '/Users/varunk/models/models--Salesforce--Llama-xLAM-2-8b-fc-r/snapshots/d10ff9682648d264a364a61505af5c0e172895f8',
                  'description': 'BYOC - Salesforce/Llama-xLAM-2-8b-fc-r',
                  'model_arch': 'llama',
                  'model_name': 'Salesforce--Llama-xLAM-2-8b-fc-r',
                  'param_count': 8,
                  'publisher': 'Salesforce',
                  'seq_length': 131072,
                  'vocab_size': 128256}]}


## 4. Upload checkpoint

In [11]:
results = byoc.upload_checkpoints()

2025-05-02 09:22:36,226 [INFO] Model with name 'Salesforce--Llama-xLAM-2-8b-fc-r' not found
2025-05-02 09:22:36,286 [INFO] App with name 'eb0aaad1-694f-41b6-958a-b974737635c4' found with id eb0aaad1-694f-41b6-958a-b974737635c4
2025-05-02 09:22:36,288 [INFO] running snapi upload command:
 snapi import model create --model-name Salesforce--Llama-xLAM-2-8b-fc-r --app eb0aaad1-694f-41b6-958a-b974737635c4 --source-type LOCAL --source-path /Users/varunk/models/models--Salesforce--Llama-xLAM-2-8b-fc-r/snapshots/d10ff9682648d264a364a61505af5c0e172895f8 --model-arch llama --parameter-count 8b --sequence-length 131072 --vocab-size 128256 -ni --publisher Salesforce --description BYOC - Salesforce/Llama-xLAM-2-8b-fc-r --ignore-transformers-version
This could take a while
2025-05-02 10:16:21,179 [INFO] Model with name 'Salesforce--Llama-xLAM-2-8b-fc-r' found with id 53432eb3-5f7c-41da-94fe-d0273c6b223a
2025-05-02 10:16:21,188 [INFO] Model checkpoint with name 'Salesforce--Llama-xLAM-2-8b-fc-r' crea

In [12]:
results

[{'name': 'Salesforce--Llama-xLAM-2-8b-fc-r',
  'id': '53432eb3-5f7c-41da-94fe-d0273c6b223a'}]

You can check the status of the checkpoints after the upload process

In [14]:
status = byoc.get_checkpoints_status(results[0]["id"])
status[["status", "progress"]]

2025-05-02 10:24:20,797 [INFO] model 53432eb3-5f7c-41da-94fe-d0273c6b223a status: 
 {'model_id': '53432eb3-5f7c-41da-94fe-d0273c6b223a', 'status': 'Available', 'progress': 100, 'stage': 'convert', 'status_code': 200, 'headers': {'access-control-allow-headers': 'Accept, Content-Type, Content-Length, Accept-Encoding, Authorization, ResponseType, Access-Control-Allow-Origin', 'access-control-allow-methods': 'GET, POST, PATCH, DELETE', 'access-control-allow-origin': 'https://sjc3-e9.sambanova.net', 'content-security-policy': "default-src 'self'", 'content-type': 'application/json', 'permissions-policy': 'none', 'referrer-policy': 'no-referrer', 'strict-transport-security': 'max-age=31536000; includeSubDomains, max-age=31536000; includeSubDomains', 'x-content-type-options': 'nosniff', 'x-correlation-id': 'a3e64860-6b75-42a7-aed9-a2cec6d38abb', 'x-frame-options': 'DENY', 'date': 'Fri, 02 May 2025 17:24:20 GMT', 'x-envoy-upstream-service-time': '38', 'server': 'istio-envoy', 'content-encoding

TypeError: list indices must be integers or slices, not list